In [28]:
import pandas as pd
import numpy as np
import os
from tqdm import tqdm
from sklearn.metrics import fbeta_score

In [29]:
data_dir = '../input'
!ls ../input

'ls' is not recognized as an internal or external command,
operable program or batch file.


In [30]:
labels_str = 'agriculture, artisinal_mine, bare_ground, blooming, blow_down, clear, cloudy, conventional_mine, cultivation, habitation, haze, partly_cloudy, primary, road, selective_logging, slash_burn, water'
labels = labels_str.split(', ')
label_map = {x: labels.index(x) for x in labels}

In [31]:
def tags_to_vec(tags):
    tags_list = tags.split(' ')
    vec = np.zeros(17)
    for t in tags_list:
        vec[label_map[t]] = 1
    return vec

In [32]:
train_label = pd.read_csv(os.path.join(data_dir, 'train_v2.csv'))
train_label['y_true'] = train_label.tags.apply(tags_to_vec)
print(train_label.shape)
train_label.head()

(40479, 3)


,image_name,tags,y_true
0,train_0,haze primary,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,train_1,agriculture clear primary water,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
2,train_2,clear primary,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
3,train_3,clear primary,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
4,train_4,agriculture clear habitation primary road,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."


In [33]:
pred = pd.read_csv(os.path.join(data_dir, '../output/keras_pred_model-0.14135_on_train_BM0.csv'))
pred['y_pred'] = pred.tags.apply(tags_to_vec)
print(pred.shape)
pred.head()

(40479, 3)


,image_name,tags,y_pred
0,train_0,clear haze primary,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
1,train_1,agriculture clear cultivation habitation prima...,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, ..."
2,train_10,agriculture clear cultivation habitation prima...,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, ..."
3,train_100,agriculture clear cultivation habitation prima...,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, ..."
4,train_1000,partly_cloudy primary,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [34]:
merged_df = pd.merge(left=train_label, right=pred, left_on='image_name', right_on='image_name')

In [35]:
merged_df.head()

,image_name,tags_x,y_true,tags_y,y_pred
0,train_0,haze primary,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",clear haze primary,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
1,train_1,agriculture clear primary water,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",agriculture clear cultivation habitation prima...,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 1.0, ..."
2,train_2,clear primary,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",clear primary,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
3,train_3,clear primary,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",clear primary,"[0.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."
4,train_4,agriculture clear habitation primary road,"[1.0, 0.0, 0.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ...",agriculture bare_ground clear habitation prima...,"[1.0, 0.0, 1.0, 0.0, 0.0, 1.0, 0.0, 0.0, 0.0, ..."


In [36]:
y_true = []
y_pred = []
for idx, row in tqdm(merged_df.iterrows(), total=merged_df.shape[0]):
    y_true.append(row['y_true'])
    y_pred.append(row['y_pred'])

100%|█████████████████████████████████████████████████████████████████████████| 40479/40479 [00:02<00:00, 17230.66it/s]


In [37]:
y_true = np.stack(y_true, axis=0)
y_pred = np.stack(y_pred, axis=0)

In [38]:
fbeta_score(y_true, y_pred, beta=2, average='samples')

0.87745992576522047